In [8]:
# download the dataset and normalise

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

#load the consolidated dataset
df = pd.read_csv(r'C:\Users\alexm\FX_project\2000_2024\data2.csv')

# remove extra rows and columns
df = df.iloc[200:5383, 0:62]

# reformat Local time variable as a datetime
from datetime import datetime

df['Date'] = df['Date'].str[6:10] +'-'+ df['Date'].str[3:5] +'-'+ df['Date'].str[0:2]
df['Date'] = pd.to_datetime(df['Date'])

# normalise data

df_norm = df.iloc[:, 5:]
df_dates = df.iloc[:, :5]

for col in df_norm.columns:
    df_norm[col] = (df_norm[col] - df_norm[col].mean())/ df_norm[col].std()

x = df_norm.to_numpy()
y = df_dates.to_numpy()
cols = df.columns
z = np.append(y,x, axis=1)

df_norm2 = pd.DataFrame(z, columns=cols)

# add an integer series to replace date for use in the ANN
df_norm2['DateID'] = range(df_norm2.shape[0])
first_column = df_norm2.pop('DateID') 
df_norm2.insert(0, 'DateID', first_column)

df_norm2.iloc[:,2:] = df_norm2.iloc[:,2:].astype(float)

df_norm2.info()

C:\Users\alexm\AppData\Local\Temp\ipykernel_4208\4247761149.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\alexm\FX_project\2000_2024\data2.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5183 entries, 0 to 5182
Data columns (total 63 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DateID           5183 non-null   int64         
 1   Date             5183 non-null   datetime64[ns]
 2   Day              5183 non-null   float64       
 3   Month            5183 non-null   float64       
 4   Year             5183 non-null   float64       
 5   Close_Mid        5183 non-null   float64       
 6   Volume_Bid       5183 non-null   float64       
 7   Volume_Ask       5183 non-null   float64       
 8   Volume_Tot       5183 non-null   float64       
 9   Close_Sprd       5183 non-null   float64       
 10  5D-MA            5183 non-null   float64       
 11  10D-MA           5183 non-null   float64       
 12  20D-MA           5183 non-null   float64       
 13  50D-MA           5183 non-null   float64       
 14  200D-MA          5183 non-null   float64

In [9]:
# check for stationarity of mean and variance. Two options:

# 1. track variance across sub time-series
df_ts = df_norm2['Close_Mid']

n = 10
m = 5

# create an empty list to contain the time series
series = []

# create elements of this list that are n + m series starting and time = 0, then time = 1 etc.
for i in range(len(df_ts) - n - m +1):
    series.append(df_ts[i:i+n+m])

# convert series to an array
series = np.array(series)

# calculate mean and variance of each time series
cols = ['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14']

df_series = pd.DataFrame(series, columns=cols)
df_series['Variance'] = df_series.var(axis=1)
df_series['Mean'] = df_series[['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14']].mean(axis=1)

# export to view in Excel
df_series.to_csv('data_varCheck_v3.csv')

df_series['Mean'].head()


0    1.879571
1    1.880129
2    1.878323
3    1.874248
4    1.869645
Name: Mean, dtype: float64

In [10]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss

X = df_norm2.values[:,5]

# 2. apply the Augmented Dickey-Fuller (ADF) test
# https://machinelearningmastery.com/time-series-data-stationary-python/

result_adf = adfuller(X)
print('ADF Statistic: %f' % result_adf[0])
print('p-value: %f' % result_adf[1])
print('Lags used: %f' % result_adf[2])
print('Observations: %f' % result_adf[3])
print('Critical Values:')
for key, value in result_adf[4].items():
    print('\t%s: %.3f' % (key, value))
print('Info criterion: %f' % result_adf[5])

# p statistic of 0.52 indicates the dataset is non-stationary so we expect some time dependency in the means and variances
# that can possibly be learned. The moving mean or variance dynamic is expected from my knowledge of FX option pricing


# 3. apply the Kwiatkowski-Phillips-Schmidt-Shin (KPSS) test

result_kpss = kpss(X)
print('\nKPSS Statistic: %f' % result_kpss[0])
print('p-value: %f' % result_kpss[1])
print('Lags: %f' % result_adf[2])
print('Critical Values:')
for key, value in result_kpss[3].items():
    print('\t%s: %.3f' % (key, value))
    
# KPSS p-value of 0.01 (small) indicates that the series is non-stationary but not to a very significant level
# KPSS looks for trends. This result indicates there is a trend
# https://www.machinelearningplus.com/time-series/kpss-test-for-stationarity/?utm_content=cmp-true

ADF Statistic: -1.519318
p-value: 0.523934
Lags used: 1.000000
Observations: 5181.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
Info criterion: -33943.044711

KPSS Statistic: 9.674500
p-value: 0.010000
Lags: 1.000000
Critical Values:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739


C:\Users\alexm\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
